<h1> Transfer Learning Model for Capstone Project </h1>

Downloading weights related to ImageNet for ResNet50:

In [ ]:
# 이 파일을 기준으로 서버에 올려주세요! 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install jsonlines
!pip install openreview-py
!pip install pdf2image

     |████████████████████████████████| 356 kB 12.3 MB/s 
     |████████████████████████████████| 162 kB 76.0 MB/s 
     |████████████████████████████████| 412 kB 64.8 MB/s 
     |████████████████████████████████| 803 kB 61.5 MB/s 
     |████████████████████████████████| 2.0 MB 56.4 MB/s 
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136835 sha256=91c4d3accf6a7eb5a7cb269174f190c1506df893494e0a2b5df678b7f662e506
  Stored in directory: /root/.cache/pip/wheels/f1/8a/f5/33ee79d4473eb201b519fa40f989b842e373237395a3421f52
Successfully built pylatexenc
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, 

In [ ]:
#@title

import numpy as np

import torch
from torch import nn
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torchvision.models as models

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # 학습 환경 설정

The custom Model that we would be using for the project:

In [ ]:
def mse(one, two):
  return ((one - two) ** 2).mean()
  
def rms(one, two):
  return np.sqrt(mse(one, two))

<h1>Data Preperation </h1>

In [ ]:
#@title
from torch.utils.data import DataLoader

from torchvision import transforms
import openreview
import torchvision
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset
import glob
from PIL import Image
import jsonlines
import os
from tqdm import tqdm
import gc

In [ ]:
from torch.utils.data import Dataset
from PIL import Image
import jsonlines
import os
from tqdm import tqdm
import cv2
import numpy as np
import torch

class PaperChannelDataSet(Dataset):
    def __init__(self, overall_image_path, transform=None):
        print("initialize data sets")
        self.transform = transform
        rating_dict = {}
        self.paper_path_list = list()
        self.score_list = list()
        years = ["2021"]
        for year in years:
            year_image_path = overall_image_path+"iclr"+year+"/"
            with jsonlines.open(f"iclr{year}_metadata.jsonl") as read_file:
                for line in read_file.iter():
                    rating_dict[line['forum']] = line['rating']
            input_paths = os.listdir(year_image_path)
            for one_file_image_path in tqdm(input_paths, desc="make data set"):
                paper_path = year_image_path + one_file_image_path + "/"
                self.paper_path_list.append(paper_path)
                rating = rating_dict[one_file_image_path]
                self.score_list.append(rating)

    def __len__(self):
        return len(self.paper_path_list)

    def __getitem__(self, idx):
        lst = list()
        paper_path = self.paper_path_list[idx]  # paper_path : dataset/image/iclr2021/_0kaDkv3dVf/
        label = self.score_list[idx]
        for i in range(9):
            binary_file = f"{paper_path}{i+1}.bin"   # binary_file = dataset/image/iclr2021/_0kaDkv3dVf/3.bin 
            with open(binary_file, 'rb') as f:
                data = f.read()
            encoded_img = np.fromstring(data, dtype=np.uint8)
            img = cv2.imdecode(encoded_img, cv2.IMREAD_COLOR) 
            if self.transform is not None:
                img = self.transform(img)
            lst.append(img) 
        tensor = torch.cat(lst, 0) 
        return tensor, label

In [ ]:
image_path =  "drive/Shareddrives/소종-논문/test/" 
dataset_file_name = 'iclr2021_dataset.pt'

In [ ]:
def make_save_data_set(image_path, dataset_file_name):
  print("start to make data set")
  transform = transforms.Compose([
      # transforms.Resize((224, 224)),
      transforms.ToTensor()
  ])

  dataset = PaperChannelDataSet(image_path, transform=transform)
  print(f"data set length: {dataset.__len__()}")

  torch.save(dataset, image_path + dataset_file_name)
  print("save data sets")

In [ ]:
make_save_data_set(image_path, dataset_file_name)

start to make data set
initialize data sets


make data set: 100%|██████████| 56/56 [00:00<00:00, 131072.00it/s]

data set length: 56
save data sets


In [ ]:
dataset = torch.load(image_path + dataset_file_name)
dataloader = DataLoader(dataset)

<h1> Training and Testing </h1>

In [ ]:
from torch.utils.data.dataset import random_split

dataset = torch.load(image_path + dataset_file_name)

train_size = int(0.8 * len(dataset))
print("train size:", train_size)

test_size = len(dataset) - train_size
print("test size:", test_size)

train_dataset, test_dataset = random_split(dataset, [train_size,test_size])
batch_size = 16
train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

train size: 44
test size: 12


In [ ]:
def create_model(input_channel, using_transfer_learning=True):
  assert(input_channel % 3 == 0, "The number of channels needs to be a multiple of 3")

  resnet50 = models.resnet50(pretrained=using_transfer_learning).to(device)

  if input_channel != 3:
    old_layer = resnet50.conv1
    
    # Creating a new Conv2d layer
    new_layer = nn.Conv2d(in_channels=input_channel, # 3 -> input_channel
                      out_channels=old_layer.out_channels, # 64
                      kernel_size=old_layer.kernel_size, # (7, 7)
                      stride=old_layer.stride, # (2, 2)
                      padding=old_layer.padding, # (3, 3)
                      bias=old_layer.bias) # False

    if using_transfer_learning:
      for channel in range(3, input_channel, 3):
        new_layer.weight[:, channel:channel+3, :, :] = old_layer.weight[:, 0:3, : :].clone()

    with torch.no_grad():
      # Copying the weights from the old to the new layer
      new_layer.weight[:, 0:3, :, :] = old_layer.weight.clone()
      
    new_layer.weight = nn.Parameter(new_layer.weight, requires_grad=(not using_transfer_learning))
  
    resnet50.conv1 = new_layer

  if using_transfer_learning:
    for param in resnet50.parameters():
      param.requires_grad = False

  model = nn.Sequential(
      resnet50,
      nn.Flatten(),
      nn.Linear(1000, 256),
      nn.ReLU(),
      nn.Linear(256, 32),
      nn.ReLU(),
      nn.Linear(32, 1)
  ).to(device)

  return model

<ipython-input-17-b097515042bf>:2: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(input_channel % 3 == 0, "The number of channels needs to be a multiple of 3")


In [ ]:
img_width, img_height, input_channel = 224, 224, 27

non_transfer_learning_model = create_model(input_channel, using_transfer_learning=False)

print(non_transfer_learning_model.eval())

Sequential(
  (0): ResNet(
    (conv1): Conv2d(27, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0

In [ ]:
from torchsummary import summary

summary(non_transfer_learning_model, (27, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]          84,672
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [ ]:
loss_fn = nn.MSELoss().to(device)
optimizer = torch.optim.Adam(non_transfer_learning_model.parameters(), lr=0.0001)

In [ ]:
total_params = sum(param.numel() for param in non_transfer_learning_model.parameters() if param.requires_grad)
print(total_params)

25896809


In [ ]:
import time
import datetime
from numpy import vstack

all_test_losses = []

def getLoss(pred, labels):
  return loss_fn(pred.to(torch.float32), labels.to(torch.float32))

def train():
    gc.collect()
    torch.cuda.empty_cache()

    # loss_function=loss_fn

    print("start train")
    print("train size:", train_size)
    print("test size:", test_size)

    trained_number = 0
    training_loss = 0
    total_start = time.time()

    for epoch in range(1, num_epochs + 1):
      epoch_start = time.time()

      for i, data in enumerate(train_dataloader):
        # i_start = time.time()
        
        # train dataloader 로 불러온 데이터에서 이미지와 라벨을 분리
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.type(torch.LongTensor) 
        labels = labels.to(device)

        # process_time = time.time()

        # 이전 batch에서 계산된 가중치를 초기화
        optimizer.zero_grad() 
        # forward + back propagation 연산
        outputs = non_transfer_learning_model(inputs).squeeze()
        
        # i_loss = time.time()

        # train_loss = loss_fn(outputs.to(torch.float32), labels.to(torch.float32))
        train_loss = getLoss(outputs, labels)

        train_loss.backward()
        optimizer.step()

        training_loss += train_loss.item()
        trained_number += labels.size(0)

        # i_end = time.time()

        # print("process_time:", process_time - i_start, "pred_time:", i_loss - process_time,"loss_time:",i_end - i_loss)
        
        # if i % 20 == 0:
        #   print("i is:", i)
        
        if i % 50 == 0:
          print(f"epoch {epoch}: {(trained_number/train_size)*100}% train finish")
        # break
    
      print(f"epoch {epoch} train finish")
      # test accuracy 계산
      total = 0
      correct = 0
      # test_loss_list = list()
      test_loss_total = 0

      for i, data in enumerate(test_dataloader):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.type(torch.LongTensor)
        labels = labels.to(device)

        # 결과값 연산
        outputs = non_transfer_learning_model(inputs).squeeze()

        total += labels.size(0)
        correct += (abs(outputs - labels)<0.5).sum().item()
        # test_loss = loss_fn(outputs.to(torch.float32), labels.to(torch.float32)).item()
        test_loss = getLoss(outputs, labels)

        # test_loss_list.append(test_loss)
        test_loss_total += test_loss

        all_test_losses.append(test_loss)
        
        # Calculating random MSE:

        if i%20==0:
          print(f"epoch {epoch} {total/test_size*100}% test finish")
        # break

      # 학습 결과 출력
      # print('Epoch: %d/%d, Train loss: %.6f, Test loss: %.6f, Accuracy: %.2f' %(epoch, num_epochs, train_loss.item(), sum(test_loss_list)/len(test_loss_list), 100*correct/total))
      print('Epoch: %d/%d, Train loss: %.6f, Test loss: %.6f, Accuracy: %.2f' %(epoch, num_epochs, train_loss.item(), test_loss_total/labels.size(0), 100*correct/total))

      epoch_elapsed_time = time.time() - epoch_start
      epoch_elapsed_time_list = str(datetime.timedelta(seconds=epoch_elapsed_time)).split(".")
      total_elapsed_time = time.time() - total_start
      total_elapsed_time_list = str(datetime.timedelta(seconds=total_elapsed_time)).split(".")
      print(f"Epoch {epoch} Elapsed time is {epoch_elapsed_time_list[0]}")  
      print(f"Total Elapsed time is {total_elapsed_time_list[0]}")  

In [ ]:
num_epochs = 1

all_test_losses = []

train()

start train
train size: 44
test size: 12
img shape: torch.Size([3, 224, 224])
img shape: torch.Size([3, 224, 224])
img shape: torch.Size([3, 224, 224])
img shape: torch.Size([3, 224, 224])
img shape: torch.Size([3, 224, 224])
img shape: torch.Size([3, 224, 224])
img shape: torch.Size([3, 224, 224])
img shape: torch.Size([3, 224, 224])
img shape: torch.Size([3, 224, 224])
img shape: torch.Size([3, 224, 224])
img shape: torch.Size([3, 224, 224])
img shape: torch.Size([3, 224, 224])
img shape: torch.Size([3, 224, 224])
img shape: torch.Size([3, 224, 224])
img shape: torch.Size([3, 224, 224])
img shape: torch.Size([3, 224, 224])
img shape: torch.Size([3, 224, 224])
img shape: torch.Size([3, 224, 224])
img shape: torch.Size([3, 224, 224])
img shape: torch.Size([3, 224, 224])
img shape: torch.Size([3, 224, 224])
img shape: torch.Size([3, 224, 224])
img shape: torch.Size([3, 224, 224])
img shape: torch.Size([3, 224, 224])
img shape: torch.Size([3, 224, 224])
img shape: torch.Size([3, 224, 224

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


img shape: torch.Size([3, 224, 224])
img shape: torch.Size([3, 224, 224])
img shape: torch.Size([3, 224, 224])
img shape: torch.Size([3, 224, 224])
img shape: torch.Size([3, 224, 224])
img shape: torch.Size([3, 224, 224])
img shape: torch.Size([3, 224, 224])
img shape: torch.Size([3, 224, 224])
img shape: torch.Size([3, 224, 224])
img shape: torch.Size([3, 224, 224])
img shape: torch.Size([3, 224, 224])
img shape: torch.Size([3, 224, 224])
img shape: torch.Size([3, 224, 224])
img shape: torch.Size([3, 224, 224])
img shape: torch.Size([3, 224, 224])
img shape: torch.Size([3, 224, 224])
img shape: torch.Size([3, 224, 224])
img shape: torch.Size([3, 224, 224])
img shape: torch.Size([3, 224, 224])
img shape: torch.Size([3, 224, 224])
img shape: torch.Size([3, 224, 224])
img shape: torch.Size([3, 224, 224])
img shape: torch.Size([3, 224, 224])
img shape: torch.Size([3, 224, 224])
img shape: torch.Size([3, 224, 224])
img shape: torch.Size([3, 224, 224])
img shape: torch.Size([3, 224, 224])
i

In [ ]:
transfer_learnt_model = create_model(input_channel, True)
print(transfer_learnt_model.eval())

TypeError: ignored

In [ ]:
total_params_transfer_learning = sum(param.numel() for param in transfer_learnt_model.parameters() if param.requires_grad)
print(total_params_transfer_learning)

NameError: ignored

In [ ]:
all_test_losses = []

def train_transfer_learnt_model():
    gc.collect()
    torch.cuda.empty_cache()

    # loss_function=loss_fn

    print("start train")
    print("train size:", train_size)
    print("test size:", test_size)

    trained_number = 0
    training_loss = 0
    total_start = time.time()

    for epoch in range(1, num_epochs + 1):
      epoch_start = time.time()

      for i, data in enumerate(train_dataloader):
        # i_start = time.time()
        
        # train dataloader 로 불러온 데이터에서 이미지와 라벨을 분리
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.type(torch.LongTensor) 
        labels = labels.to(device)

        # process_time = time.time()

        # 이전 batch에서 계산된 가중치를 초기화
        optimizer.zero_grad() 
        # forward + back propagation 연산
        outputs = transfer_learnt_model(inputs).squeeze()
        
        # i_loss = time.time()

        # train_loss = loss_fn(outputs.to(torch.float32), labels.to(torch.float32))
        train_loss = getLoss(outputs, labels)

        train_loss.backward()
        optimizer.step()

        training_loss += train_loss.item()
        trained_number += labels.size(0)

        # i_end = time.time()

        # print("process_time:", process_time - i_start, "pred_time:", i_loss - process_time,"loss_time:",i_end - i_loss)
        
        # if i % 20 == 0:
        #   print("i is:", i)
        
        if i % 50 == 0:
          print(f"epoch {epoch}: {(trained_number/train_size)*100}% train finish")
        # break
    
      print(f"epoch {epoch} train finish")
      # test accuracy 계산
      total = 0
      correct = 0
      # test_loss_list = list()
      test_loss_total = 0

      for i, data in enumerate(test_dataloader):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.type(torch.LongTensor)
        labels = labels.to(device)

        # 결과값 연산
        outputs = transfer_learnt_model(inputs).squeeze()

        total += labels.size(0)
        correct += (abs(outputs - labels)<0.5).sum().item()
        # test_loss = loss_fn(outputs.to(torch.float32), labels.to(torch.float32)).item()
        test_loss = getLoss(outputs, labels)

        # test_loss_list.append(test_loss)
        test_loss_total += test_loss

        all_test_losses.append(test_loss)
        
        # Calculating random MSE:


        if i%20==0:
          print(f"epoch {epoch} {total/test_size*100}% test finish")
        # break

      # 학습 결과 출력
      # print('Epoch: %d/%d, Train loss: %.6f, Test loss: %.6f, Accuracy: %.2f' %(epoch, num_epochs, train_loss.item(), sum(test_loss_list)/len(test_loss_list), 100*correct/total))
      print('Epoch: %d/%d, Train loss: %.6f, Test loss: %.6f, Accuracy: %.2f' %(epoch, num_epochs, train_loss.item(), test_loss_total/labels.size(0), 100*correct/total))

      epoch_elapsed_time = time.time() - epoch_start
      epoch_elapsed_time_list = str(datetime.timedelta(seconds=epoch_elapsed_time)).split(".")
      total_elapsed_time = time.time() - total_start
      total_elapsed_time_list = str(datetime.timedelta(seconds=total_elapsed_time)).split(".")
      print(f"Epoch {epoch} Elapsed time is {epoch_elapsed_time_list[0]}")  
      print(f"Total Elapsed time is {total_elapsed_time_list[0]}")  